<a href="https://colab.research.google.com/github/PP-lib/bot_local/blob/master/Distortion_plot_simple.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import datetime
import math
import csv
import glob

In [0]:
from google.colab import drive
drive.mount('./gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at ./gdrive


In [0]:

# 同じフォルダのCSVファイルの一覧を取得
files = sorted(glob.glob('./gdrive/My Drive/Bot_materials/materials/data/data1/*.csv'))

# CSVファイルの数を取得
file_number = len(files)

# CSVファイルの中身を読み出して、リスト形式にまとめる
csv_list = []
for file in files:
    csv_list.append(pd.read_csv(file))
# リスト形式にまとめたCSVファイルを結合
merge_csv = pd.concat(csv_list)

df_origin = merge_csv
df_origin

,exec_date,buy_child_order_acceptance_id,exec_date.1,id,price,sell_child_order_acceptance_id,side,size
0,2020-02-14 21:33:07.890912+00:00,JRF20200214-123254-568817,2020-02-14 21:33:07.890912+00:00,1577298839,1183588.0,JRF20200214-123307-459369,SELL,0.001
1,2020-02-14 21:33:07.890912+00:00,JRF20200214-123247-628727,2020-02-14 21:33:07.890912+00:00,1577298840,1183493.0,JRF20200214-123307-459369,SELL,0.009
2,2020-02-14 21:33:07.890912+00:00,JRF20200214-123254-568817,2020-02-14 21:33:07.890912+00:00,1577298839,1183588.0,JRF20200214-123307-459369,SELL,0.001
3,2020-02-14 21:33:07.890912+00:00,JRF20200214-123247-628727,2020-02-14 21:33:07.890912+00:00,1577298840,1183493.0,JRF20200214-123307-459369,SELL,0.009
4,2020-02-14 21:33:11.300225200+00:00,JRF20200214-123247-628727,2020-02-14 21:33:11.300225200+00:00,1577298841,1183493.0,JRF20200214-123311-793238,SELL,0.136
...,...,...,...,...,...,...,...,...
113323,2020-02-15 03:59:59.075288100+00:00,JRF20200214-185957-541778,2020-02-15 03:59:59.075288100+00:00,1577376180,1193890.0,JRF20200214-185958-096077,SELL,0.010
113324,2020-02-15 03:59:59.908980100+00:00,JRF20200214-185957-541778,2020-02-15 03:59:59.908980100+00:00,1577376182,1193890.0,JRF20200214-185959-683425,SELL,0.020
113325,2020-02-15 03:59:59.908980100+00:00,JRF20200214-185957-541778,2020-02-15 03:59:59.908980100+00:00,1577376182,1193890.0,JRF20200214-185959-683425,SELL,0.020
113326,2020-02-15 04:00:01.048236900+00:00,JRF20200214-190000-603630,2020-02-15 04:00:01.048236900+00:00,1577376184,1194000.0,JRF20200214-185758-733921,BUY,0.040


In [0]:
#時間のまるめ
df_T = df_origin['exec_date.1']
df_origin['exec_date.1'] = pd.to_datetime(df_origin['exec_date.1']).dt.round("1s")
df_origin

,exec_date,buy_child_order_acceptance_id,exec_date.1,id,price,sell_child_order_acceptance_id,side,size
0,2020-02-14 21:33:07.890912+00:00,JRF20200214-123254-568817,2020-02-14 21:33:08+00:00,1577298839,1183588.0,JRF20200214-123307-459369,SELL,0.001
1,2020-02-14 21:33:07.890912+00:00,JRF20200214-123247-628727,2020-02-14 21:33:08+00:00,1577298840,1183493.0,JRF20200214-123307-459369,SELL,0.009
2,2020-02-14 21:33:07.890912+00:00,JRF20200214-123254-568817,2020-02-14 21:33:08+00:00,1577298839,1183588.0,JRF20200214-123307-459369,SELL,0.001
3,2020-02-14 21:33:07.890912+00:00,JRF20200214-123247-628727,2020-02-14 21:33:08+00:00,1577298840,1183493.0,JRF20200214-123307-459369,SELL,0.009
4,2020-02-14 21:33:11.300225200+00:00,JRF20200214-123247-628727,2020-02-14 21:33:11+00:00,1577298841,1183493.0,JRF20200214-123311-793238,SELL,0.136
...,...,...,...,...,...,...,...,...
113323,2020-02-15 03:59:59.075288100+00:00,JRF20200214-185957-541778,2020-02-15 03:59:59+00:00,1577376180,1193890.0,JRF20200214-185958-096077,SELL,0.010
113324,2020-02-15 03:59:59.908980100+00:00,JRF20200214-185957-541778,2020-02-15 04:00:00+00:00,1577376182,1193890.0,JRF20200214-185959-683425,SELL,0.020
113325,2020-02-15 03:59:59.908980100+00:00,JRF20200214-185957-541778,2020-02-15 04:00:00+00:00,1577376182,1193890.0,JRF20200214-185959-683425,SELL,0.020
113326,2020-02-15 04:00:01.048236900+00:00,JRF20200214-190000-603630,2020-02-15 04:00:01+00:00,1577376184,1194000.0,JRF20200214-185758-733921,BUY,0.040


In [0]:
#丸め時間あたりの価格平均算出前処理
df_T_price = pd.concat([df_T, df_origin['price']], axis=1)
#df_T_price

In [0]:
#1秒間の平均価格
df_pivot_price = df_origin.pivot_table(values = ['price'],
               index = ['exec_date.1'],
               aggfunc = 'mean').round()
df_pivot_price = df_pivot_price.rename(columns={'price':'price_mean(1s)'}).reset_index()
#df_pivot_price_shift = df_pivot_price.shift(1) 
df_pivot_price_shift_10s = df_pivot_price['price_mean(1s)'].shift(10).rename("price_mean(1s)_shift(10s)")
df_pivot_price_shift_70s = df_pivot_price['price_mean(1s)'].shift(70).rename("price_mean(1s)_shift(10s)")
df_pivot_price_shift_60s = df_pivot_price['price_mean(1s)'].shift(60).rename("price_mean(1s)_shift(0s)")
df_pivot_price_difference = (df_pivot_price_shift_70s - df_pivot_price_shift_60s).rename("price_difference")
df_pivot_price = pd.concat([df_pivot_price,df_pivot_price_shift_10s, df_pivot_price_difference], axis=1)

#df_pivot_price = df_pivot_price.rename(columns={'2':'price_mean(1)'}).reset_index()


df_pivot_price

,exec_date.1,price_mean(1s),price_mean(1s)_shift(10s),price_difference
0,2020-02-14 21:33:08+00:00,1183540.0,NaN,NaN
1,2020-02-14 21:33:11+00:00,1183434.0,NaN,NaN
2,2020-02-14 21:33:12+00:00,1183298.0,NaN,NaN
3,2020-02-14 21:33:15+00:00,1182989.0,NaN,NaN
4,2020-02-14 21:33:18+00:00,1182797.0,NaN,NaN
...,...,...,...,...
11558,2020-02-15 03:59:55+00:00,1193756.0,1192929.0,1048.0
11559,2020-02-15 03:59:57+00:00,1193863.0,1192958.0,144.0
11560,2020-02-15 03:59:59+00:00,1193945.0,1193378.0,22.0
11561,2020-02-15 04:00:00+00:00,1193890.0,1193731.0,59.0


In [0]:
#移動平均の追加
df_origin = pd.merge(df_origin,df_pivot_price,on='exec_date.1')
df_origin

,exec_date,buy_child_order_acceptance_id,exec_date.1,id,price,sell_child_order_acceptance_id,side,size,price_mean(1s),price_mean(1s)_shift(10s),price_difference
0,2020-02-14 21:33:07.890912+00:00,JRF20200214-123254-568817,2020-02-14 21:33:08+00:00,1577298839,1183588.0,JRF20200214-123307-459369,SELL,0.001,1183540.0,NaN,NaN
1,2020-02-14 21:33:07.890912+00:00,JRF20200214-123247-628727,2020-02-14 21:33:08+00:00,1577298840,1183493.0,JRF20200214-123307-459369,SELL,0.009,1183540.0,NaN,NaN
2,2020-02-14 21:33:07.890912+00:00,JRF20200214-123254-568817,2020-02-14 21:33:08+00:00,1577298839,1183588.0,JRF20200214-123307-459369,SELL,0.001,1183540.0,NaN,NaN
3,2020-02-14 21:33:07.890912+00:00,JRF20200214-123247-628727,2020-02-14 21:33:08+00:00,1577298840,1183493.0,JRF20200214-123307-459369,SELL,0.009,1183540.0,NaN,NaN
4,2020-02-14 21:33:11.300225200+00:00,JRF20200214-123247-628727,2020-02-14 21:33:11+00:00,1577298841,1183493.0,JRF20200214-123311-793238,SELL,0.136,1183434.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
113323,2020-02-15 03:59:59.075288100+00:00,JRF20200214-185957-541778,2020-02-15 03:59:59+00:00,1577376180,1193890.0,JRF20200214-185958-096077,SELL,0.010,1193945.0,1193378.0,22.0
113324,2020-02-15 03:59:59.908980100+00:00,JRF20200214-185957-541778,2020-02-15 04:00:00+00:00,1577376182,1193890.0,JRF20200214-185959-683425,SELL,0.020,1193890.0,1193731.0,59.0
113325,2020-02-15 03:59:59.908980100+00:00,JRF20200214-185957-541778,2020-02-15 04:00:00+00:00,1577376182,1193890.0,JRF20200214-185959-683425,SELL,0.020,1193890.0,1193731.0,59.0
113326,2020-02-15 04:00:01.048236900+00:00,JRF20200214-190000-603630,2020-02-15 04:00:01+00:00,1577376184,1194000.0,JRF20200214-185758-733921,BUY,0.040,1194000.0,1193378.0,364.0


In [0]:
df_origin['size'] = df_origin['size']/0.001
#df_origin['size'] = df_origin['size'].values
df_origin['size'] = np.log10(df_origin['size']).astype(float)
#df_log_size = pd.DataFrame(log_size)
#df_log_size = df_log_size.rename(columns={0:'log_size'}).astype(float)
df_origin

,exec_date,buy_child_order_acceptance_id,exec_date.1,id,price,sell_child_order_acceptance_id,side,size,price_mean(1s),price_mean(1s)_shift(10s),price_difference
0,2020-02-14 21:33:07.890912+00:00,JRF20200214-123254-568817,2020-02-14 21:33:08+00:00,1577298839,1183588.0,JRF20200214-123307-459369,SELL,0.000000,1183540.0,NaN,NaN
1,2020-02-14 21:33:07.890912+00:00,JRF20200214-123247-628727,2020-02-14 21:33:08+00:00,1577298840,1183493.0,JRF20200214-123307-459369,SELL,0.954243,1183540.0,NaN,NaN
2,2020-02-14 21:33:07.890912+00:00,JRF20200214-123254-568817,2020-02-14 21:33:08+00:00,1577298839,1183588.0,JRF20200214-123307-459369,SELL,0.000000,1183540.0,NaN,NaN
3,2020-02-14 21:33:07.890912+00:00,JRF20200214-123247-628727,2020-02-14 21:33:08+00:00,1577298840,1183493.0,JRF20200214-123307-459369,SELL,0.954243,1183540.0,NaN,NaN
4,2020-02-14 21:33:11.300225200+00:00,JRF20200214-123247-628727,2020-02-14 21:33:11+00:00,1577298841,1183493.0,JRF20200214-123311-793238,SELL,2.133539,1183434.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
113323,2020-02-15 03:59:59.075288100+00:00,JRF20200214-185957-541778,2020-02-15 03:59:59+00:00,1577376180,1193890.0,JRF20200214-185958-096077,SELL,1.000000,1193945.0,1193378.0,22.0
113324,2020-02-15 03:59:59.908980100+00:00,JRF20200214-185957-541778,2020-02-15 04:00:00+00:00,1577376182,1193890.0,JRF20200214-185959-683425,SELL,1.301030,1193890.0,1193731.0,59.0
113325,2020-02-15 03:59:59.908980100+00:00,JRF20200214-185957-541778,2020-02-15 04:00:00+00:00,1577376182,1193890.0,JRF20200214-185959-683425,SELL,1.301030,1193890.0,1193731.0,59.0
113326,2020-02-15 04:00:01.048236900+00:00,JRF20200214-190000-603630,2020-02-15 04:00:01+00:00,1577376184,1194000.0,JRF20200214-185758-733921,BUY,1.602060,1194000.0,1193378.0,364.0


In [0]:
df_origin['side'] = df_origin['side'].replace({'BUY':'-1', 'SELL':'1'})
#side_ex = pd.DataFrame(side_ex).astype(int)
df_origin

,exec_date,buy_child_order_acceptance_id,exec_date.1,id,price,sell_child_order_acceptance_id,side,size,price_mean(1s),price_mean(1s)_shift(10s),price_difference
0,2020-02-14 21:33:07.890912+00:00,JRF20200214-123254-568817,2020-02-14 21:33:08+00:00,1577298839,1183588.0,JRF20200214-123307-459369,1,0.000000,1183540.0,NaN,NaN
1,2020-02-14 21:33:07.890912+00:00,JRF20200214-123247-628727,2020-02-14 21:33:08+00:00,1577298840,1183493.0,JRF20200214-123307-459369,1,0.954243,1183540.0,NaN,NaN
2,2020-02-14 21:33:07.890912+00:00,JRF20200214-123254-568817,2020-02-14 21:33:08+00:00,1577298839,1183588.0,JRF20200214-123307-459369,1,0.000000,1183540.0,NaN,NaN
3,2020-02-14 21:33:07.890912+00:00,JRF20200214-123247-628727,2020-02-14 21:33:08+00:00,1577298840,1183493.0,JRF20200214-123307-459369,1,0.954243,1183540.0,NaN,NaN
4,2020-02-14 21:33:11.300225200+00:00,JRF20200214-123247-628727,2020-02-14 21:33:11+00:00,1577298841,1183493.0,JRF20200214-123311-793238,1,2.133539,1183434.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
113323,2020-02-15 03:59:59.075288100+00:00,JRF20200214-185957-541778,2020-02-15 03:59:59+00:00,1577376180,1193890.0,JRF20200214-185958-096077,1,1.000000,1193945.0,1193378.0,22.0
113324,2020-02-15 03:59:59.908980100+00:00,JRF20200214-185957-541778,2020-02-15 04:00:00+00:00,1577376182,1193890.0,JRF20200214-185959-683425,1,1.301030,1193890.0,1193731.0,59.0
113325,2020-02-15 03:59:59.908980100+00:00,JRF20200214-185957-541778,2020-02-15 04:00:00+00:00,1577376182,1193890.0,JRF20200214-185959-683425,1,1.301030,1193890.0,1193731.0,59.0
113326,2020-02-15 04:00:01.048236900+00:00,JRF20200214-190000-603630,2020-02-15 04:00:01+00:00,1577376184,1194000.0,JRF20200214-185758-733921,-1,1.602060,1194000.0,1193378.0,364.0


In [0]:
df_origin['size'] = df_origin['side'].astype(int)*df_origin['size']

#size_ex = pd.DataFrame(size_ex)
#size_ex = size_ex.rename(columns={0:'log_size'})


#df_size_std = size_ex.rolling(60).std()
#df_size_std = pd.DataFrame(df_size_std)
#df_size_std = df_size_std.rename(columns={'log_size': 'size_std(60)'})


#df_all = pd.concat([df_price_add, size_ex, df_size_std], axis=1).dropna()

#df_all

df_origin

,exec_date,buy_child_order_acceptance_id,exec_date.1,id,price,sell_child_order_acceptance_id,side,size,price_mean(1s),price_mean(1s)_shift(10s),price_difference
0,2020-02-14 21:33:07.890912+00:00,JRF20200214-123254-568817,2020-02-14 21:33:08+00:00,1577298839,1183588.0,JRF20200214-123307-459369,1,0.000000,1183540.0,NaN,NaN
1,2020-02-14 21:33:07.890912+00:00,JRF20200214-123247-628727,2020-02-14 21:33:08+00:00,1577298840,1183493.0,JRF20200214-123307-459369,1,0.954243,1183540.0,NaN,NaN
2,2020-02-14 21:33:07.890912+00:00,JRF20200214-123254-568817,2020-02-14 21:33:08+00:00,1577298839,1183588.0,JRF20200214-123307-459369,1,0.000000,1183540.0,NaN,NaN
3,2020-02-14 21:33:07.890912+00:00,JRF20200214-123247-628727,2020-02-14 21:33:08+00:00,1577298840,1183493.0,JRF20200214-123307-459369,1,0.954243,1183540.0,NaN,NaN
4,2020-02-14 21:33:11.300225200+00:00,JRF20200214-123247-628727,2020-02-14 21:33:11+00:00,1577298841,1183493.0,JRF20200214-123311-793238,1,2.133539,1183434.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
113323,2020-02-15 03:59:59.075288100+00:00,JRF20200214-185957-541778,2020-02-15 03:59:59+00:00,1577376180,1193890.0,JRF20200214-185958-096077,1,1.000000,1193945.0,1193378.0,22.0
113324,2020-02-15 03:59:59.908980100+00:00,JRF20200214-185957-541778,2020-02-15 04:00:00+00:00,1577376182,1193890.0,JRF20200214-185959-683425,1,1.301030,1193890.0,1193731.0,59.0
113325,2020-02-15 03:59:59.908980100+00:00,JRF20200214-185957-541778,2020-02-15 04:00:00+00:00,1577376182,1193890.0,JRF20200214-185959-683425,1,1.301030,1193890.0,1193731.0,59.0
113326,2020-02-15 04:00:01.048236900+00:00,JRF20200214-190000-603630,2020-02-15 04:00:01+00:00,1577376184,1194000.0,JRF20200214-185758-733921,-1,-1.602060,1194000.0,1193378.0,364.0


In [0]:
#1秒間の平均価格
df_heat_plot = df_origin.pivot_table(values = ['size'],
               index = ['exec_date.1'], columns = ['price']).fillna(0)
               

df_heat_plot




size            ...                    
price                     1179000.0 1179001.0  ... 1196000.0 1196100.0
exec_date.1                                    ...                    
2020-02-14 21:33:08+00:00       0.0       0.0  ...       0.0       0.0
2020-02-14 21:33:11+00:00       0.0       0.0  ...       0.0       0.0
2020-02-14 21:33:12+00:00       0.0       0.0  ...       0.0       0.0
2020-02-14 21:33:15+00:00       0.0       0.0  ...       0.0       0.0
2020-02-14 21:33:18+00:00       0.0       0.0  ...       0.0       0.0
...                             ...       ...  ...       ...       ...
2020-02-15 03:59:55+00:00       0.0       0.0  ...       0.0       0.0
2020-02-15 03:59:57+00:00       0.0       0.0  ...       0.0       0.0
2020-02-15 03:59:59+00:00       0.0       0.0  ...       0.0       0.0
2020-02-15 04:00:00+00:00       0.0       0.0  ...       0.0       0.0
2020-02-15 04:00:01+00:00       0.0       0.0  ...       0.0       0.0

[11563 rows x 10089 columns]

In [0]:
#1秒間の価格差
df_threshold = df_origin.pivot_table(values = ['price_difference'],
               index = ['exec_date.1']).dropna()

df_threshold

,price_difference
exec_date.1,
2020-02-14 21:35:13+00:00,819.0
2020-02-14 21:35:14+00:00,786.0
2020-02-14 21:35:15+00:00,683.0
2020-02-14 21:35:17+00:00,303.0
2020-02-14 21:35:19+00:00,111.0
...,...
2020-02-15 03:59:55+00:00,1048.0
2020-02-15 03:59:57+00:00,144.0
2020-02-15 03:59:59+00:00,22.0


In [0]:
import matplotlib.pyplot as plt
import seaborn as sns



In [0]:
ups = sorted(glob.glob('./gdrive/My Drive/Bot_materials/materials/plots/up/*.png'))
print("up:", len(ups))

downs = sorted(glob.glob('./gdrive/My Drive/Bot_materials/materials/plots/down/*.png'))
print("down:", len(downs))

unknowns = sorted(glob.glob('./gdrive/My Drive/Bot_materials/materials/plots/unknown/*.png'))
print("unknown:", len(unknowns))


file_number=ups + downs + unknowns

len(file_number)
# if len(df_heat_plot) == (len(ups) + len(downs) + len(unknowns) + plot_size):
#     print(True,"data:", len(df_heat_plot), "png:",len(ups) + len(downs) + len(unknowns) + plot_size)

# else:
#     print(False,"data:", len(df_heat_plot), "png:",len(ups) + len(downs) + len(unknowns) + plot_size)


up: 99
down: 138
unknown: 5


242

In [0]:


#for処理部分
#サンプリングサイズ
plot_size = 60
i = plot_size
threshold_plus= 2.5
threshold_minus = -2.5

file_number=len(ups + downs + unknowns)

for i in range(plot_size, len(df_heat_plot), 1):
    sns.heatmap(df_heat_plot.iloc[:i,:], cmap="seismic")#,vmin=size_min,vmax=size_max)#, square=True)

    if df_threshold['price_difference'].iat[i] > threshold_plus:

        plt.savefig('./gdrive/My Drive/Bot_materials/materials/plots/up/{}.png'.format(i))
        plt.close('all')

    elif df_threshold['price_difference'].iat[i] < threshold_minus:
        
        plt.savefig('./gdrive/My Drive/Bot_materials/materials/plots/down/{}.png'.format(i))
        plt.close('all')

    else:
        plt.savefig('./gdrive/My Drive/Bot_materials/materials/plots/unknown/{}.png'.format(i))
        plt.close('all')
        

# 新しいセクション

In [0]:
len(df_heat_plot) 

In [0]:
len(df_pivot_price.dropna())

In [0]:
import keras
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.core import Dense, Dropout, Activation, Flatten
import numpy as np
from sklearn.model_selection import train_test_split
from PIL import Image
import glob

folder = ['up','down']
image_size_X = 240
image_size_Y = 240

# 画像の読み込み、リサイズ、ndarrayへの格納 
X = []
Y = []
for index, name in enumerate(folder):
    dir = './gdrive/My Drive/Bot_materials/materials/plots/' + name
    files = glob.glob(dir + '/*.png')
    for i, file in enumerate(files):
        image = Image.open(file)
        image = image.convert('RGB')
        image = image.resize((image_size_X, image_size_Y))
        data = np.asarray(image)
        X.append(data)
        Y.append(index)
 
X = np.array(X)
Y = np.array(Y)

# RGBデータの正規化
X = X.astype('float32')
X = X / 255.0

# 正解ラベルの形式を変換
Y = np_utils.to_categorical(Y, 3)

# 訓練データとテストデータの分配(2割をテストデータへ)  
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.20)

# CNNのパラメータ設定
model = Sequential()
 
model.add(Conv2D(32, (3, 3), padding='same',input_shape=X_train.shape[1:])) # レイヤー1  
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3))) # レイヤー2  
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
 
model.add(Conv2D(64, (3, 3), padding='same'))  # レイヤー3  
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))   # レイヤー4  
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
 
model.add(Flatten())   # レイヤー5
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(3))   # レイヤー6  
model.add(Activation('softmax'))
 
# コンパイル
model.compile(loss='categorical_crossentropy',optimizer='SGD',metrics=['accuracy'])

#訓練
model.fit(X_train, Y_train,batch_size=32, epochs=100)

#評価 & 評価結果出力
score = model.evaluate(X_test, Y_test)
print('loss=',score[0])
print('accuracy=',score[1])

In [0]:
# モデルの保存
open('and.json',"w").write(model.to_json())

# 学習済みの重みを保存
model.save_weights('and_weight.hdf5')

In [0]:
import numpy as np
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras.optimizers import Adam
import keras.callbacks
import keras.backend.tensorflow_backend as KTF
import tensorflow as tf
import os.path
import json


model.save('./gdrive/My Drive/Bot_materials/materials/data/data1/model.h5', include_optimizer=False)
model.save_weights(os.path.join('./gdrive/My Drive/Bot_materials/materials/data/data1/cnn_model_weights.hdf5'))

# model_json_str = model.to_json()
# open('mnist_mlp_model.json', 'w').write(model_json_str)
# model.save_weights('mnist_mlp_weights.h5');


In [0]:
import numpy as np
import cv2
from keras.models import model_from_json
from keras.preprocessing.image import load_img, img_to_array

MODEL_ARC_PATH = "and.json"

# JSONファイルからモデルのアーキテクチャを得る
model_arc_str = open(MODEL_ARC_PATH, 'utf-8', 'ignore').read()
model = model_from_json(model_arc_str)

# モデル構成の確認
model.summary()

In [0]:
from keras.models import load_model
import numpy as np
from keras.preprocessing.image import img_to_array, load_img

png_name = '137'
model_file_name='./gdrive/My Drive/Bot_materials/materials/data/data1/cnn_model_weights1.hdf5'

model=load_model('./gdrive/My Drive/Bot_materials/materials/data/data1/model.h5')

img_path = ('./gdrive/My Drive/Bot_materials/materials/plots/down/' + png_name + '.png')
img = img_to_array(load_img(img_path, target_size=(240,240)))
img_nad = img_to_array(img)/255
img_nad = img_nad[None, ...]

label=['up','down']
pred = model.predict(img_nad, batch_size=1, verbose=0)
score = np.max(pred)
pred_label = label[np.argmax(pred[0])]
print('name:',pred_label)
print('score:',score)

name: down
score: 0.55760896


/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:310: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [0]:
pip install h5py

In [0]:
print('./gdrive/My Drive/Bot_materials/materials/params/cnn_model_weights.h5')
